# SEO Content Quality & Duplicate Detector

This notebook implements the full pipeline:

1. Load raw SEO dataset from CSV.
2. Extract clean text from HTML (title + body).
3. Engineer SEO-related features:
   - word_count
   - sentence_count
   - Flesch Reading Ease (readability)
4. Build TF-IDF representations for pages.
5. Detect duplicate / near-duplicate URLs using cosine similarity.
6. Create synthetic quality labels (High / Medium / Low).
7. Train a RandomForest classifier to predict quality.
8. Save processed files and the trained model.

In [2]:
# 1. Imports and paths

from pathlib import Path
import re

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from scipy.sparse import save_npz
import pickle

# Project directories (relative)
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / "data"
MODELS_DIR = BASE_DIR / "models"

DATA_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)

# Your absolute path to the raw CSV
ABS_INPUT_PATH = Path("data/data.csv")

ABS_INPUT_PATH, DATA_DIR, MODELS_DIR

(PosixPath('data/data.csv'),
 PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/data'),
 PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/models'))

## 2. Load dataset from absolute path

We read the raw data from the given absolute path on the local machine and also
save a copy into `./data/data.csv` for reproducibility.

In [4]:
# 2. Load raw dataset

if not ABS_INPUT_PATH.exists():
    raise FileNotFoundError(f"data.csv not found at: {ABS_INPUT_PATH}")

df_raw = pd.read_csv(ABS_INPUT_PATH)
df_raw.columns = [c.strip().lower() for c in df_raw.columns]  # normalize columns

print(df_raw.shape)
df_raw.head()

(81, 2)


,url,html_content
0,https://www.cm-alliance.com/cybersecurity-blog,"<!doctype html><!--[if lt IE 7]> <html class=""..."
1,https://www.varonis.com/blog/cybersecurity-tips,"<!doctype html><html lang=""en""><head>\n <me..."
2,https://www.cisecurity.org/insights/blog/11-cy...,<!DOCTYPE html><html data-unhead-vue-server-re...
3,https://www.cisa.gov/topics/cybersecurity-best...,"\n\n<!DOCTYPE html>\n<html lang=""en"" dir=""ltr""..."
4,https://www.qnbtrust.bank/Resources/Learning-C...,NaN


In [5]:
# Save a copy inside the project data folder
project_raw_path = DATA_DIR / "data.csv"
df_raw.to_csv(project_raw_path, index=False)
project_raw_path

PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/data/data.csv')

## 3. Quick EDA

Check basic statistics and missing values to understand the dataset.

In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           81 non-null     object
 1   html_content  69 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


In [8]:
df_raw.isna().mean() * 100  # percentage of missing values per column

url              0.000000
html_content    14.814815
dtype: float64

## 4. Helper functions

We define reusable text-processing and readability functions, plus HTML parsing logic.

In [10]:
def clean_text(s: str) -> str:
    """Remove weird spaces and collapse multiple spaces."""
    if not isinstance(s, str):
        return ""
    s = s.replace("\xa0", " ")
    s = re.sub(r"\s+", " ", s)
    return s.strip()


def sentence_tokenize(text: str):
    """Very simple sentence splitter based on punctuation."""
    if not isinstance(text, str):
        return []
    return [s.strip() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]


def word_tokenize(text: str):
    """Split text into alphanumeric tokens."""
    if not isinstance(text, str):
        return []
    return re.findall(r"[A-Za-z0-9']+", text.lower())


def count_syllables(word: str) -> int:
    """Rough syllable counter – good enough for Flesch score."""
    word = word.lower()
    if len(word) <= 3:
        return 1
    vowels = "aeiouy"
    count = 0
    prev_vowel = False
    for ch in word:
        is_v = ch in vowels
        if is_v and not prev_vowel:
            count += 1
        prev_vowel = is_v
    if word.endswith("e"):
        count = max(1, count - 1)
    return max(1, count)


def flesch_reading_ease(text: str) -> float:
    """
    Compute Flesch Reading Ease score.
    Higher = easier to read.
    """
    words = word_tokenize(text)
    sentences = sentence_tokenize(text)
    if len(words) == 0 or len(sentences) == 0:
        return 0.0

    syllables = sum(count_syllables(w) for w in words)
    W = len(words)
    S = len(sentences)
    score = 206.835 - 1.015 * (W / S) - 84.6 * (syllables / W)
    return round(score, 2)


def extract_text_from_html(html: str):
    """
    Extract title + main body text from HTML using BeautifulSoup.
    Heuristic:
    1. Drop script/style/noscript.
    2. Prefer <main> or <article>.
    3. Fallback to concatenated <p> tags.
    4. Last fallback: whole page text.
    """
    if not isinstance(html, str) or not html.strip():
        return "", ""

    soup = BeautifulSoup(html, "html.parser")

    for t in soup(["script", "style", "noscript"]):
        t.extract()

    title = soup.title.get_text(separator=" ", strip=True) if soup.title else ""

    candidates = []
    for tag_name in ["main", "article"]:
        tag = soup.find(tag_name)
        if tag:
            candidates.append(tag.get_text(separator=" ", strip=True))

    if not candidates:
        ps = [p.get_text(separator=" ", strip=True) for p in soup.find_all("p")]
        if len(ps) >= 2:
            candidates.append(" ".join(ps))
        else:
            candidates.append(soup.get_text(separator=" ", strip=True))

    body = max(candidates, key=len) if candidates else ""
    return clean_text(title), clean_text(body)

## 5. Extract main content from HTML

For each row, we parse the `html_content`, extract the `<title>` and the main body,
and compute the raw `word_count`.

In [12]:
titles = []
bodies = []
word_counts = []

for _, row in df_raw.iterrows():
    html = row.get("html_content", "")
    title, body = extract_text_from_html(html)
    titles.append(title)
    bodies.append(body)
    word_counts.append(len(word_tokenize(body)))

extracted = pd.DataFrame({
    "url": df_raw["url"],
    "title": titles,
    "body_text": bodies,
    "word_count": word_counts
})

extracted.head()

,url,title,body_text,word_count
0,https://www.cm-alliance.com/cybersecurity-blog,Cyber Security Blog,Cyber Crisis Tabletop Exercise Cyber Security ...,332
1,https://www.varonis.com/blog/cybersecurity-tips,Top 10 Cybersecurity Awareness Tips: How to St...,Blog Privacy & Compliance Top 10 Cybersecurity...,1778
2,https://www.cisecurity.org/insights/blog/11-cy...,11 Cyber Defense Tips to Stay Secure at Work a...,Home Insights Blog Posts 11 Cyber Defense Tips...,1085
3,https://www.cisa.gov/topics/cybersecurity-best...,Cybersecurity Best Practices | Cybersecurity a...,Cybersecurity Best Practices CISA provides inf...,844
4,https://www.qnbtrust.bank/Resources/Learning-C...,,,0


In [13]:
extracted_path = DATA_DIR / "extracted_content.csv"
extracted.to_csv(extracted_path, index=False)
extracted_path

PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/data/extracted_content.csv')

## 6. Feature engineering

We compute:

- sentence_count
- flesch_reading_ease
- thin content flag (`is_thin`)
- a synthetic quality_label (High / Medium / Low)

In [15]:
sentence_counts = [len(sentence_tokenize(t)) for t in extracted["body_text"]]
readability_scores = [flesch_reading_ease(t) for t in extracted["body_text"]]

def label_quality(word_count: int, fre: float) -> str:
    """
    Simple rule-based quality label:
    - High: long & comfortably readable
    - Low: very short or very hard to read
    - Medium: in-between
    """
    if (word_count > 1500) and (50 <= fre <= 70):
        return "High"
    if (word_count < 500) or (fre < 30):
        return "Low"
    return "Medium"

features = pd.DataFrame({
    "url": extracted["url"],
    "word_count": extracted["word_count"],
    "sentence_count": sentence_counts,
    "flesch_reading_ease": readability_scores
})

features["is_thin"] = (features["word_count"] < 500).astype(int)
features["quality_label"] = [
    label_quality(wc, fre) for wc, fre in zip(features["word_count"], features["flesch_reading_ease"])
]

features.head()

,url,word_count,sentence_count,flesch_reading_ease,is_thin,quality_label
0,https://www.cm-alliance.com/cybersecurity-blog,332,7,-10.25,1,Low
1,https://www.varonis.com/blog/cybersecurity-tips,1778,94,42.51,0,Medium
2,https://www.cisecurity.org/insights/blog/11-cy...,1085,72,56.26,0,Medium
3,https://www.cisa.gov/topics/cybersecurity-best...,844,27,-5.02,0,Low
4,https://www.qnbtrust.bank/Resources/Learning-C...,0,0,0.00,1,Low


In [16]:
features_path = DATA_DIR / "features.csv"
features.to_csv(features_path, index=False)
features_path

PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/data/features.csv')

## 7. TF-IDF embeddings & duplicate detection

We represent each page using TF-IDF vectors on the `body_text` and
compute cosine similarity. Pairs with similarity ≥ 0.80 are treated
as potential duplicates / near-duplicates.

In [18]:
texts_for_tfidf = extracted["body_text"].fillna("").astype(str).tolist()

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words="english"
)

X_tfidf = tfidf_vectorizer.fit_transform(texts_for_tfidf)
X_tfidf.shape

(81, 5000)

In [19]:
emb_path = DATA_DIR / "tfidf_embeddings.npz"
save_npz(emb_path, X_tfidf)
emb_path

PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/data/tfidf_embeddings.npz')

In [20]:
sim_matrix = cosine_similarity(X_tfidf)
n = sim_matrix.shape[0]
threshold = 0.8

pairs = []
for i in range(n):
    for j in range(i + 1, n):
        sim = sim_matrix[i, j]
        if sim >= threshold:
            pairs.append({
                "url1": extracted.loc[i, "url"],
                "url2": extracted.loc[j, "url"],
                "similarity": round(float(sim), 4)
            })

duplicates_df = pd.DataFrame(pairs)
duplicates_df.head()

,url1,url2,similarity
0,https://en.wikipedia.org/wiki/SD-WAN,https://www.fortinet.com/resources/cyberglossa...,0.8714
1,https://www.microsoft.com/en-us/security/busin...,https://www.zscaler.com/resources/security-ter...,0.8494
2,https://www.forbes.com/advisor/business/what-i...,https://blog.hubspot.com/marketing/what-is-dig...,0.8278


In [21]:
dup_path = DATA_DIR / "duplicates.csv"
duplicates_df.to_csv(dup_path, index=False)
dup_path, len(duplicates_df)

(PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/data/duplicates.csv'),
 3)

In [22]:
dup_path = DATA_DIR / "duplicates.csv"
duplicates_df.to_csv(dup_path, index=False)

summary = {
    "total_pages": int(len(extracted)),
    "duplicate_pairs": int(len(duplicates_df)),
    "thin_pages": int(features["is_thin"].sum()),
    "thin_pct": float(features["is_thin"].mean() * 100)
}

dup_path, summary

(PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/data/duplicates.csv'),
 {'total_pages': 81,
  'duplicate_pairs': 3,
  'thin_pages': 28,
  'thin_pct': 34.5679012345679})

## 8. Train quality classifier

We train a RandomForest classifier to predict `quality_label` from:

- word_count
- sentence_count
- flesch_reading_ease

We also compare with a simple baseline that uses only `word_count`.

In [24]:
X_model = features[["word_count", "sentence_count", "flesch_reading_ease"]].values
y = features["quality_label"].values

# Baseline using only word_count (for your report)
baseline_pred = np.where(
    features["word_count"] > 1000,
    "High",
    np.where(features["word_count"] < 500, "Low", "Medium")
)

baseline_acc = accuracy_score(y, baseline_pred)
baseline_acc

0.49382716049382713

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X_model, y, test_size=0.3, random_state=42, stratify=y
)

clf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
cm = confusion_matrix(y_test, y_pred, labels=["Low", "Medium", "High"])
report = classification_report(
    y_test, y_pred, labels=["Low", "Medium", "High"], zero_division=0
)

acc, f1

(0.96, 0.9543859649122808)

In [26]:
print("Baseline accuracy (word_count rule):", baseline_acc)
print("Model accuracy:", acc)
print("Model weighted F1:", f1)
print("Confusion matrix (Low, Medium, High):\n", cm)
print("\nClassification report:\n", report)

# Feature importance
feature_names = ["word_count", "sentence_count", "flesch_reading_ease"]
importances = clf.feature_importances_

print("\nTop features (by importance):")
for name, imp in sorted(zip(feature_names, importances), key=lambda x: -x[1]):
    print(f"- {name}: {imp:.3f}")

Baseline accuracy (word_count rule): 0.49382716049382713
Model accuracy: 0.96
Model weighted F1: 0.9543859649122808
Confusion matrix (Low, Medium, High):
 [[14  0  0]
 [ 0  9  0]
 [ 0  1  1]]

Classification report:
               precision    recall  f1-score   support

         Low       1.00      1.00      1.00        14
      Medium       0.90      1.00      0.95         9
        High       1.00      0.50      0.67         2

    accuracy                           0.96        25
   macro avg       0.97      0.83      0.87        25
weighted avg       0.96      0.96      0.95        25


Top features (by importance):
- flesch_reading_ease: 0.443
- sentence_count: 0.290
- word_count: 0.267


In [27]:
model_path = MODELS_DIR / "quality_model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(clf, f)

model_path

PosixPath('/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/models/quality_model.pkl')

# Real-Time Analysis Demo

Requirements:
- Implement `analyze_url(url)` function in this notebook.
- For any given URL:
  - Scrape the page with `requests`:
    - Set a User-Agent header
    - Add 1–2s delay (polite crawling)
    - Handle errors (timeouts / non-200 responses)
  - Extract main content using the same HTML parser.
  - Compute features:
    - word_count
    - sentence_count
    - readability (Flesch score)
  - Predict:
    - rule-based quality label
    - model-based quality label (RandomForest)
  - Compute similarity to existing dataset pages:
    - Use TF-IDF vectorizer fitted earlier (`tfidf_vectorizer`)
    - Return top similar URLs above a threshold (e.g., 0.5)
- Return a Python dict and print it as JSON.

In [29]:
import requests
import time
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; SEO-Detector/1.0; +https://example.com)"
}

def analyze_url(url: str, sim_threshold: float = 0.5, delay: float = 1.5):
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        time.sleep(delay)  # polite delay
        if resp.status_code != 200:
            return {"url": url, "error": f"HTTP {resp.status_code}"}
    except Exception as e:
        return {"url": url, "error": str(e)}

    title, body = extract_text_from_html(resp.text)
    wc = len(word_tokenize(body))
    sc = len(sentence_tokenize(body))
    fre = flesch_reading_ease(body)
    rule_label = label_quality(wc, fre)

    try:
        X_new = np.array([[wc, sc, fre]])
        model_label = clf.predict(X_new)[0]
    except Exception:
        model_label = rule_label

    new_vec = tfidf_vectorizer.transform([body])
    sims = cosine_similarity(new_vec, X_tfidf).ravel()
    top_idx = sims.argsort()[::-1][:10]

    similar_pages = [
        {"url": extracted.loc[i, "url"], "similarity": round(float(sims[i]), 4)}
        for i in top_idx if sims[i] >= sim_threshold
    ]

    return {
        "url": url,
        "title": title,
        "word_count": wc,
        "sentence_count": sc,
        "readability": fre,
        "rule_quality_label": rule_label,
        "model_quality_label": model_label,
        "is_thin": wc < 500,
        "similar_pages": similar_pages
    }

# Test
test_url = "https://en.wikipedia.org/wiki/Data_science"
result = analyze_url(test_url)
print(json.dumps(result, indent=2))

{
  "url": "https://en.wikipedia.org/wiki/Data_science",
  "title": "Data science - Wikipedia",
  "word_count": 2698,
  "sentence_count": 288,
  "readability": 43.3,
  "rule_quality_label": "Medium",
  "model_quality_label": "Medium",
  "is_thin": false,
  "similar_pages": [
    {
      "url": "https://realpython.com/tutorials/data-science/",
      "similarity": 0.5444
    }
  ]
}


# Streamlit App Integration

In this final section, I generate a modular **Streamlit application** directly from the
notebook. The app reuses:

- the processed dataset (`data/extracted_content.csv`, `data/features.csv`)
- the trained RandomForest model (`models/quality_model.pkl`)
- the same feature engineering and quality rules as the notebook

The code below creates the following structure:

```text
streamlit_app/
├── app.py
├── utils/
│   ├── parser.py
│   ├── features.py
│   └── scorer.py
└── models/
    └── quality_model.pkl

In [74]:
from pathlib import Path
import shutil

# Base is the same as earlier in the notebook
BASE_DIR = Path().resolve()

# Create Streamlit app folders
STREAMLIT_DIR = BASE_DIR / "streamlit_app"
UTILS_DIR = STREAMLIT_DIR / "utils"
STREAMLIT_MODELS_DIR = STREAMLIT_DIR / "models"

STREAMLIT_DIR.mkdir(exist_ok=True)
UTILS_DIR.mkdir(exist_ok=True)
STREAMLIT_MODELS_DIR.mkdir(exist_ok=True)

print("Created folders:")
print(STREAMLIT_DIR)
print(UTILS_DIR)
print(STREAMLIT_MODELS_DIR)

# Copy trained model into streamlit_app/models
src_model = MODELS_DIR / "quality_model.pkl"   # created earlier in Section 4
dst_model = STREAMLIT_MODELS_DIR / "quality_model.pkl"
shutil.copy2(src_model, dst_model)

print("Copied model →", dst_model)

Created folders:
/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/streamlit_app
/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/streamlit_app/utils
/Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/streamlit_app/models
Copied model → /Users/ajaykumark/Documents/UniArchive/MDS- Semesters/seo-content-detector/streamlit_app/models/quality_model.pkl


In [54]:
from pathlib import Path

parser_code = '''import re
import requests
from bs4 import BeautifulSoup


HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/119.0.0.0 Safari/537.36"
    )
}


def fetch_and_parse(url: str, timeout: int = 10) -> str | None:
    """
    Fetch raw HTML from a URL. Returns HTML or None on failure.
    """
    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
        if resp.status_code != 200:
            return None
        return resp.text
    except Exception:
        return None


def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.replace("\\xa0", " ")
    return re.sub(r"\\s+", " ", s).strip()


def extract_text_from_html(html: str) -> tuple[str, str]:
    """
    Extract (title, body_text) from a raw HTML string using a simple heuristic.
    """
    soup = BeautifulSoup(html, "html.parser")

    for t in soup(["script", "style", "noscript"]):
        t.extract()

    title = soup.title.get_text(separator=" ", strip=True) if soup.title else ""

    candidates = []
    for tag_name in ["main", "article"]:
        tag = soup.find(tag_name)
        if tag:
            candidates.append(tag.get_text(separator=" ", strip=True))

    if not candidates:
        ps = [p.get_text(separator=" ", strip=True) for p in soup.find_all("p")]
        if len(ps) >= 2:
            candidates.append(" ".join(ps))
        else:
            candidates.append(soup.get_text(separator=" ", strip=True))

    body = max(candidates, key=len) if candidates else ""
    return clean_text(title), clean_text(body)
'''

# save the corrected file
UTILS_DIR = Path("streamlit_app/utils")
UTILS_DIR.mkdir(parents=True, exist_ok=True)
(UTILS_DIR / "parser.py").write_text(parser_code, encoding="utf-8")

print("✅ parser.py created successfully →", UTILS_DIR / "parser.py")

✅ parser.py created successfully → streamlit_app/utils/parser.py


In [56]:
from pathlib import Path

features_code = '''import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


def sentence_tokenize(text: str):
    if not isinstance(text, str):
        return []
    return [s.strip() for s in re.split(r"(?<=[.!?])\\s+", text) if s.strip()]


def word_tokenize(text: str):
    if not isinstance(text, str):
        return []
    return re.findall(r"[A-Za-z0-9']+", text.lower())


def count_syllables(word: str) -> int:
    word = word.lower()
    if len(word) <= 3:
        return 1
    vowels = "aeiouy"
    count = 0
    prev_vowel = False
    for ch in word:
        is_v = ch in vowels
        if is_v and not prev_vowel:
            count += 1
        prev_vowel = is_v
    if word.endswith("e"):
        count = max(1, count - 1)
    return max(1, count)


def flesch_reading_ease(text: str) -> float:
    words = word_tokenize(text)
    sentences = sentence_tokenize(text)
    if len(words) == 0 or len(sentences) == 0:
        return 0.0
    syllables = sum(count_syllables(w) for w in words)
    W = len(words)
    S = len(sentences)
    score = 206.835 - 1.015 * (W / S) - 84.6 * (syllables / W)
    return round(score, 2)


def embed_tfidf(texts, max_features: int = 5000):
    """
    Fit TF-IDF on texts and return (X, vectorizer).
    """
    texts = [t if isinstance(t, str) else "" for t in texts]
    vec = TfidfVectorizer(max_features=max_features, stop_words="english")
    X = vec.fit_transform(texts)
    return X, vec


def top_keywords(vec: TfidfVectorizer, X_row, top_k: int = 5):
    feature_names = np.array(vec.get_feature_names_out())
    row = X_row.toarray().ravel()
    if row.sum() == 0:
        return []
    idx = row.argsort()[-top_k:][::-1]
    return feature_names[idx].tolist()
'''

# Write file
UTILS_DIR = Path("streamlit_app/utils")
UTILS_DIR.mkdir(parents=True, exist_ok=True)
(UTILS_DIR / "features.py").write_text(features_code, encoding="utf-8")

print("✅ features.py created successfully →", UTILS_DIR / "features.py")

✅ features.py created successfully → streamlit_app/utils/features.py


In [60]:
from pathlib import Path

scorer_code = '''import pickle


def rule_quality_label(word_count: int, readability: float) -> str:
    """
    Rule-based quality label (same logic as notebook).
    """
    if (word_count > 1500) and (50 <= readability <= 70):
        return "High"
    if (word_count < 500) or (readability < 30):
        return "Low"
    return "Medium"


def load_quality_model(path):
    """
    Load trained RandomForest model from disk.
    """
    with open(path, "rb") as f:
        model = pickle.load(f)
    return model
'''

# Replace UTILS_DIR with your actual path if not already defined
UTILS_DIR = Path("streamlit_app/utils")
UTILS_DIR.mkdir(parents=True, exist_ok=True)
(UTILS_DIR / "scorer.py").write_text(scorer_code, encoding="utf-8")

print("✅ Written:", UTILS_DIR / "scorer.py")

✅ Written: streamlit_app/utils/scorer.py


In [72]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import streamlit as st
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------------------------------------
# Make sure Python can find the local 'utils' package
# -----------------------------------------------------------
APP_DIR = Path(__file__).resolve().parent
if str(APP_DIR) not in sys.path:
    sys.path.append(str(APP_DIR))

from utils.parser import fetch_and_parse, extract_text_from_html
from utils.features import (
    sentence_tokenize,
    word_tokenize,
    flesch_reading_ease,
    embed_tfidf,
)
from utils.scorer import load_quality_model, rule_quality_label


# -----------------------------------------------------------
# Paths
# -----------------------------------------------------------
BASE_DIR = Path(__file__).resolve().parents[1]
DATA_DIR = BASE_DIR / "data"
MODEL_FILE = APP_DIR / "models" / "quality_model.pkl"


# -----------------------------------------------------------
# Cached loaders
# -----------------------------------------------------------
@st.cache_data
def load_dataset():
    """
    Load the preprocessed dataset produced by the Jupyter pipeline.
    """
    extracted = pd.read_csv(DATA_DIR / "extracted_content.csv")
    features = pd.read_csv(DATA_DIR / "features.csv")
    return extracted, features


@st.cache_resource
def load_embeddings(texts):
    """
    Fit TF-IDF embeddings for similarity search.
    """
    X, vec = embed_tfidf(list(texts))
    return X, vec


@st.cache_resource
def load_model():
    """
    Load the trained RandomForest content quality model.
    """
    return load_quality_model(MODEL_FILE)


# -----------------------------------------------------------
# Streamlit config
# -----------------------------------------------------------
st.set_page_config(
    page_title="SEO Content Quality & Duplicate Detector",
    page_icon="🔎",
    layout="wide",
)

st.title("🔎 SEO Content Quality & Duplicate Detector")
st.markdown(
    """
This dashboard builds on the Jupyter notebook pipeline to:

- Analyse the quality of a single URL in real time.
- Detect near-duplicate pages inside the dataset.
- Summarise dataset-level SEO health (thin content, readability, labels).
"""
)

# Load core artefacts
extracted_df, features_df = load_dataset()
X_tfidf, vectorizer = load_embeddings(
    extracted_df["body_text"].fillna("").astype(str)
)
model = load_model()

# Sidebar
st.sidebar.header("Settings")
sim_threshold = st.sidebar.slider(
    "Similarity threshold (cosine)",
    min_value=0.5,
    max_value=0.95,
    value=0.8,
    step=0.01,
)
st.sidebar.caption(
    f"Pairs with cosine similarity ≥ {sim_threshold:.2f} are treated as potential duplicates."
)

tab_analyze, tab_duplicates, tab_summary = st.tabs(
    ["📍 Analyze URL", "🧬 Duplicate Map", "📊 Dataset Summary"]
)


# -----------------------------------------------------------
# Tab 1 – Analyze URL
# -----------------------------------------------------------
with tab_analyze:
    st.subheader("1. Real-time SEO analysis for a URL")

    url = st.text_input(
        "Enter a URL",
        placeholder="https://en.wikipedia.org/wiki/Data_science",
    )

    if st.button("Analyze URL"):
        if not url.strip():
            st.warning("Please enter a URL.")
        else:
            with st.spinner("Fetching and analyzing page..."):
                html = fetch_and_parse(url)
                if html is None:
                    st.error("Failed to fetch page (HTTP != 200 or network error).")
                else:
                    # Parse and compute features
                    title, body = extract_text_from_html(html)

                    wc = len(word_tokenize(body))
                    sc = len(sentence_tokenize(body))
                    fre = flesch_reading_ease(body)

                    rule_label = rule_quality_label(wc, fre)

                    # Model prediction
                    try:
                        X_new = np.array([[wc, sc, fre]])
                        model_label = model.predict(X_new)[0]
                    except Exception:
                        model_label = rule_label

                    # Similarity to dataset
                    new_vec = vectorizer.transform([body])
                    sims = cosine_similarity(new_vec, X_tfidf).ravel()
                    top_idx = sims.argsort()[::-1][:10]

                    similar_rows = []
                    for i in top_idx:
                        if sims[i] >= sim_threshold:
                            similar_rows.append(
                                {
                                    "similar_url": extracted_df.loc[i, "url"],
                                    "similarity": round(float(sims[i]), 4),
                                }
                            )
                    similar_df = pd.DataFrame(similar_rows)

                    # KPIs
                    c1, c2, c3 = st.columns(3)
                    c1.metric("Word Count", f"{wc}")
                    c2.metric("Sentence Count", f"{sc}")
                    c3.metric("Readability (FRE)", f"{fre}")

                    st.markdown("### Quality Assessment")
                    st.write(f"- **Rule-based label:** `{rule_label}`")
                    st.write(f"- **Model prediction:** `{model_label}`")
                    st.write(f"- **Thin content?** `{wc < 500}`")

                    st.markdown("### Similar Pages in Dataset")
                    if similar_df.empty:
                        st.info(
                            f"No similar pages above similarity {sim_threshold:.2f}."
                        )
                    else:
                        st.dataframe(similar_df, use_container_width=True)

                    st.markdown("### Extracted Content Preview")
                    st.caption(f"Title: {title}")
                    st.write(body[:2000] + ("..." if len(body) > 2000 else ""))


# -----------------------------------------------------------
# Tab 2 – Duplicate Map
# -----------------------------------------------------------
with tab_duplicates:
    st.subheader("2. Near-duplicate URLs in the dataset")

    with st.spinner("Computing cosine similarities..."):
        sim_matrix = cosine_similarity(X_tfidf)
        n = sim_matrix.shape[0]
        pairs = []
        for i in range(n):
            for j in range(i + 1, n):
                sim = sim_matrix[i, j]
                if sim >= sim_threshold:
                    pairs.append(
                        {
                            "url_1": extracted_df.loc[i, "url"],
                            "url_2": extracted_df.loc[j, "url"],
                            "similarity": round(float(sim), 4),
                        }
                    )

        dup_df = pd.DataFrame(pairs)

    if dup_df.empty:
        st.info(
            f"No duplicate pairs found with similarity ≥ {sim_threshold:.2f}. "
            "Try lowering the threshold in the sidebar."
        )
    else:
        st.write(
            f"Found **{len(dup_df)}** URL pairs with similarity ≥ {sim_threshold:.2f}."
        )
        st.dataframe(dup_df, use_container_width=True)


# -----------------------------------------------------------
# Tab 3 – Dataset Summary
# -----------------------------------------------------------
with tab_summary:
    st.subheader("3. Dataset-level SEO summary")

    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Total Pages", len(extracted_df))
    col2.metric("Avg Word Count", f"{features_df['word_count'].mean():.0f}")
    col3.metric(
        "Thin Content (%)",
        f"{(features_df['is_thin'].mean() * 100):.1f}%",
    )
    col4.metric(
        "Readable (FRE 50–70) (%)",
        f"{(features_df['flesch_reading_ease'].between(50, 70).mean() * 100):.1f}%",
    )

    st.markdown("#### Sample of engineered features")
    st.dataframe(features_df.head(20), use_container_width=True)

    st.markdown("#### Quality label distribution")
    st.bar_chart(features_df["quality_label"].value_counts())

NameError: name '__file__' is not defined